<font size="+2" color="blue">Additional results: surprise library</font>

# Practice PS06: Recommendations engines (interactions-based)

For this assignment we will build and apply an item-based and model-based collaborative filtering recommenders for movies. 

Author: <font color="white">Mario Muñoz Serrano</font>

Date: <font color="white">8/11/2022</font>

# 1. The Movies dataset

# 1.1. Load the input files

In [3]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366446 sha256=f3741a1528e3a1b452b0288089909c296ca4d4eaf35d233d6ee29b974530f4a8
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [4]:
# Leave this code as-is

import re
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from math import*
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import linear_kernel

In [5]:
import warnings
warnings.filterwarnings('ignore')

import traceback

In [6]:
# Leave this code as-is

FILENAME_MOVIES = "https://raw.githubusercontent.com/chatox/data-mining-course/master/practicum/data/movielens-25M-filtered/movies-2000s.csv"
FILENAME_RATINGS = "https://raw.githubusercontent.com/chatox/data-mining-course/master/practicum/data/movielens-25M-filtered/ratings-2000s.csv"
FILENAME_TAGS = "https://raw.githubusercontent.com/chatox/data-mining-course/master/practicum/data/movielens-25M-filtered/tags-2000s.csv"

In [7]:
# Leave this code as-is

movies = pd.read_csv(FILENAME_MOVIES, 
                    sep=',', 
                    engine='python', 
                    encoding='latin-1',
                    names=['movie_id', 'title', 'genres'])
display(movies.head(5))

ratings_raw = pd.read_csv(FILENAME_RATINGS, 
                    sep=',', 
                    encoding='latin-1',
                    engine='python',
                    names=['user_id', 'movie_id', 'rating'])
display(ratings_raw.head(5))

,movie_id,title,genres
0,2769,"Yards, The (2000)",Crime|Drama
1,3177,Next Friday (2000),Comedy
2,3190,Supernova (2000),Adventure|Sci-Fi|Thriller
3,3225,Down to You (2000),Comedy|Romance
4,3228,Wirey Spindell (2000),Comedy


,user_id,movie_id,rating
0,4,1,3.0
1,4,260,3.5
2,4,296,4.0
3,4,541,4.5
4,4,589,4.0


# 1.2. Merge the data into a single dataframe

In [8]:
# joined the three dataframes using "merge" into a single dataframe named "ratings".
ratings = pd.merge(ratings_raw, movies, how='inner', on='movie_id')
# Print the first 5 rows of the resulting dataframe,
ratings.head(5)

,user_id,movie_id,rating,title,genres
0,4,3624,2.5,Shanghai Noon (2000),Action|Adventure|Comedy|Western
1,152,3624,3.0,Shanghai Noon (2000),Action|Adventure|Comedy|Western
2,171,3624,3.5,Shanghai Noon (2000),Action|Adventure|Comedy|Western
3,276,3624,4.0,Shanghai Noon (2000),Action|Adventure|Comedy|Western
4,494,3624,3.5,Shanghai Noon (2000),Action|Adventure|Comedy|Western


In [9]:
# code from the previous practice for "find_movies" that list movies containing a keyword
def find_movies(string, df):
  

  contain_values = df[df['title'].str.contains(string)]

  if contain_values.empty:
    contain_values = df[df['title'] == string]
    

  for i, row in contain_values.iterrows():
    id = row['movie_id']
    title = row['title']

    print(f'movie_id: {id}')
  
  return id

In [10]:
# LEAVE AS-IS

# For testing, this should print '59784'
find_movies("Kung Fu Panda (2008)", movies)

movie_id: 59784


59784

In [11]:
# LEAVE AS-IS

def get_title(movie_id, movies):
    return movies[movies['movie_id'] == movie_id].title.iloc[0]

In [12]:
# LEAVE AS-IS

# For testing, should print "Kung Fu Panda (2008)")
print(get_title(59784, movies))

Kung Fu Panda (2008)


## 1.3. Count unique registers

In [13]:
# Count the number of unique users and unique movies in the ratings variable. Use unique(). Print also the total number of movies in the movies variable
print(f"""
        Number of users who have rated a movie : {len(ratings["user_id"].unique())}
        Number of movies that have been rated  : {len(ratings["movie_id"].unique())}
        Total number of movies                 : {len(movies["movie_id"].unique())}""")



        Number of users who have rated a movie : 12676
        Number of movies that have been rated  : 2049
        Total number of movies                 : 33168


# 2. Item-based Collaborative Filtering

## 2.1. Data pre-processing

In [14]:
# code to generate "rated_movies" and print the first five rows. This should have columns user_id, movie_id, rating, title
rated_movies = ratings[['user_id', 	'movie_id', 	'rating', 	'title']]


In [15]:
# code to generate "ratings_summary" and print the first 5 rows.

ratings_summary = movies[['movie_id', 'title']]

mean_group = rated_movies.groupby('movie_id')['rating'].mean()

mean_group_df = mean_group.to_frame().reset_index()

mean_group_df.rename(columns = {'rating':'ratings_mean'}, inplace = True)

count_by_group_df = rated_movies['movie_id'].value_counts().rename_axis('movie_id').reset_index(name='ratings_count')

In [16]:
# code to generate "ratings_summary" and print the first 5 rows.
ratings_summary = pd.merge(ratings_summary, mean_group_df, how='inner', on='movie_id')

ratings_summary = pd.merge(ratings_summary, count_by_group_df, how='inner', on='movie_id')

ratings_summary.head(5)

,movie_id,title,ratings_mean,ratings_count
0,2769,"Yards, The (2000)",3.122549,102
1,3177,Next Friday (2000),2.824000,125
2,3190,Supernova (2000),2.395683,139
3,3225,Down to You (2000),2.577273,110
4,3228,Wirey Spindell (2000),2.500000,2


In [17]:
def only_movies_receiving_at_least_N_ratings(ratings_summary, N, col):

  ratings_summary_N = ratings_summary[ratings_summary.ratings_count >= N]
  ratings_summary_N = ratings_summary_N.sort_values(by= col, ascending=False)

  return ratings_summary_N


In [18]:
# code to print the top 10 highest rated movies, considering only movies receiving at least 2000 ratings.
N = 2000

ratings_summary_N = only_movies_receiving_at_least_N_ratings(ratings_summary, N, 'ratings_mean')

ratings_summary_N.head(10)

,movie_id,title,ratings_mean,ratings_count
740,5618,Spirited Away (Sen to Chihiro no kamikakushi) ...,4.215216,2458
881,6016,City of God (Cidade de Deus) (2002),4.186592,2133
259,4226,Memento (2000),4.158512,4476
488,4973,"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)...",4.097234,3687
499,4993,"Lord of the Rings: The Fellowship of the Ring,...",4.092530,5944
1247,7153,"Lord of the Rings: The Return of the King, The...",4.083960,5449
849,5952,"Lord of the Rings: The Two Towers, The (2002)",4.083869,5449
1325,7361,Eternal Sunshine of the Spotless Mind (2004),4.074589,3593
188,4011,Snatch (2000),4.034259,2700
154,3949,Requiem for a Dream (2000),3.955145,2352


In [19]:
ratings_summary_N['ratings_mean'].describe()

count    46.000000
mean      3.786209
std       0.269121
min       2.970015
25%       3.658762
50%       3.837517
75%       3.937663
max       4.215216
Name: ratings_mean, dtype: float64

In [20]:
# this time consider movies receiving at least 3 ratings
N = 3

ratings_summary_N = only_movies_receiving_at_least_N_ratings(ratings_summary, N, 'ratings_mean')

ratings_summary_N.head(10)

,movie_id,title,ratings_mean,ratings_count
534,5082,"Rumor of Angels, A (2000)",4.666667,6
1778,27764,2LDK (2003),4.500000,3
1961,31954,Beautiful City (Shah-re ziba) (2004),4.400000,5
572,5224,Promises (2001),4.388889,18
1149,6775,Life and Debt (2001),4.333333,3
1952,31856,Surplus: Terrorized Into Being Consumers (2003),4.333333,3
1090,6672,War Photographer (2001),4.229167,24
740,5618,Spirited Away (Sen to Chihiro no kamikakushi) ...,4.215216,2458
1378,7563,"Discovery of Heaven, The (2001)",4.200000,5
881,6016,City of God (Cidade de Deus) (2002),4.186592,2133


In [21]:
ratings_summary_N['ratings_mean'].describe()

count    1854.000000
mean        3.150222
std         0.561589
min         0.750000
25%         2.785714
50%         3.221494
75%         3.574028
max         4.666667
Name: ratings_mean, dtype: float64

Now considering movies receiving at least 3 ratings, we can see several differences:
- The number of films taked into account is higher
- But the mean rating is lower and the variance is two times higher.

Maybe this is because people tend more to rate films that they like.

## 2.2. Compute the user-movie matrix

In [22]:
# code to generate a "user_movie" matrix by calling "pivot_table" on "rated_movies
user_movie = rated_movies.pivot_table(index = 'user_id', columns  = 'movie_id', values = 'rating' )

user_movie.head(10)

movie_id,2769,3177,3190,3225,3228,3239,3273,3275,3276,3279,...,33138,33145,33148,33150,33152,33154,33158,33162,33164,33166
user_id,,,,,,,,,,,,,,,,,,,,,
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This results in a matrix that is really large and sparse. Because most users don't see all this movies in their life. And even if someone did, not everybody rates movies. Most people simply whatch them.

# 2.3. Explore some correlations in the user-movie matrix

In [23]:
selected_movies = ['Monsters, Inc. (2001)', 'Finding Nemo (2003)', 'Talk to Her (Hable con Ella) (2002)']

ids = []
for title in selected_movies:
  print(title)
  ids.append(find_movies(title, movies))
  print('--')

Monsters, Inc. (2001)
movie_id: 4886
--
Finding Nemo (2003)
movie_id: 6377
--
Talk to Her (Hable con Ella) (2002)
movie_id: 5878
--


In [24]:
# compute and display the first 10 rows of the "ratings_selected_movies" table
smov = []
for id in ids:
  smov.append(user_movie[id].dropna())

ratings_selected_movies = pd.concat(smov, axis=1)

In [25]:
# code to compute all correlations between these three movies
pairs = [[0,1], [0,2], [1,2]]

for pair in pairs:
  print(f"Similarity between '{selected_movies[pair[0]]}' and '{selected_movies[pair[1]]}': {round(ratings_selected_movies[ids[pair[0]]].corr(ratings_selected_movies[ids[pair[1]]]), 3)}")

Similarity between 'Monsters, Inc. (2001)' and 'Finding Nemo (2003)': 0.639
Similarity between 'Monsters, Inc. (2001)' and 'Talk to Her (Hable con Ella) (2002)': 0.178
Similarity between 'Finding Nemo (2003)' and 'Talk to Her (Hable con Ella) (2002)': 0.212


We found that 'Monsters, Inc. (2001)' and 'Finding Nemo (2003)' is the only one that has a high correlation, maybe this is because they share some genres in common and they are from the same productor disney.

In [26]:
# code to create a "similar_to_pivot" series that contains the computed correlations, droping the NaNs in the series.
def create_similar_to_pivot(id_pivot, user_movie):

  similar_to_pivot = pd.DataFrame()
  similar_to_pivot['movie_id'] = user_movie.columns

  smov = []
  for id in user_movie.columns:
    smov.append(user_movie[id].dropna())

  ratings_selected = pd.concat(smov, axis=1)

  similar_to_pivot['corr'] = similar_to_pivot.apply(lambda x: ratings_selected[id_pivot].corr(ratings_selected[x['movie_id']]), axis=1)

  return similar_to_pivot


In [27]:
id_pivot = ids[0]
similar_to_pivot = create_similar_to_pivot(id_pivot, user_movie)

similar_to_pivot.head(3)

,movie_id,corr
0,2769,-0.082710
1,3177,0.210484
2,3190,0.039882


In [28]:
# code to create a "corr_with_pivot" dataframe as specified above, and to print the 10 movies 
# (rated 500 times or more) with the highest correlation with the selected movie.

N = 500

corr_with_pivot = pd.merge(ratings_summary, similar_to_pivot, how='inner', on='movie_id')

corr_with_pivot_N = only_movies_receiving_at_least_N_ratings(corr_with_pivot, N, 'corr')

corr_with_pivot_N.head(10)

,movie_id,title,ratings_mean,ratings_count,corr
462,4886,"Monsters, Inc. (2001)",3.850066,3775,1.000000
1007,6377,Finding Nemo (2003),3.862284,3765,0.639308
1575,8961,"Incredibles, The (2004)",3.867771,3320,0.546784
307,4306,Shrek (2001),3.768787,4591,0.471673
566,5218,Ice Age (2002),3.546400,1972,0.465032
654,5444,Lilo & Stitch (2002),3.576974,760,0.424311
83,3751,Chicken Run (2000),3.476384,2096,0.423167
1413,8360,Shrek 2 (2004),3.520989,2144,0.404075
192,4016,"Emperor's New Groove, The (2000)",3.573889,900,0.402423
73,3624,Shanghai Noon (2000),3.297443,1017,0.373850


In [29]:
#  What happens if you set the condition on ratings_count to a much larger value?
N = 2000

corr_with_pivot = pd.merge(ratings_summary, similar_to_pivot, how='inner', on='movie_id')

corr_with_pivot_N = only_movies_receiving_at_least_N_ratings(corr_with_pivot, N, 'corr')

corr_with_pivot_N.head(10)

,movie_id,title,ratings_mean,ratings_count,corr
462,4886,"Monsters, Inc. (2001)",3.850066,3775,1.000000
1007,6377,Finding Nemo (2003),3.862284,3765,0.639308
1575,8961,"Incredibles, The (2004)",3.867771,3320,0.546784
307,4306,Shrek (2001),3.768787,4591,0.471673
83,3751,Chicken Run (2000),3.476384,2096,0.423167
1413,8360,Shrek 2 (2004),3.520989,2144,0.404075
1421,8368,Harry Potter and the Prisoner of Azkaban (2004),3.809971,2397,0.362380
469,4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,3.678509,2843,0.361623
1036,6539,Pirates of the Caribbean: The Curse of the Bla...,3.779241,3950,0.356322
807,5816,Harry Potter and the Chamber of Secrets (2002),3.657751,2374,0.339883


In [30]:
# What happens if you set it to a much smaller value?
N = 3

corr_with_pivot = pd.merge(ratings_summary, similar_to_pivot, how='inner', on='movie_id')

corr_with_pivot_N = only_movies_receiving_at_least_N_ratings(corr_with_pivot, N, 'corr')

corr_with_pivot_N.head(10)

,movie_id,title,ratings_mean,ratings_count,corr
961,6292,Marion Bridge (2002),3.166667,3,1.0
462,4886,"Monsters, Inc. (2001)",3.850066,3775,1.0
119,3880,"Ballad of Ramblin' Jack, The (2000)",3.678571,14,1.0
183,3995,Boys Life 3 (2000),3.100000,5,1.0
1175,6877,Girls Will Be Girls (2003),3.333333,3,1.0
714,5537,Satin Rouge (2002),3.642857,7,1.0
1775,27754,Foolproof (2003),4.000000,4,1.0
1512,8829,Benji: Off the Leash! (2004),3.000000,4,1.0
1200,6931,Mail Order Bride (2003),3.333333,3,1.0
1687,27506,Hukkle (2002),3.750000,4,1.0


# 2.4. Implement the item-based recommendations

In [31]:
# code to compute all correlations between columns (movies) in the matrix user_movie. Store this in "item_similarity"
smov = []
for id in user_movie.columns:
  smov.append(user_movie[id].dropna())
  
ratings_selected = pd.concat(smov, axis=1)

item_similarity = ratings_selected.corr()

item_similarity.head(5)

,2769,3177,3190,3225,3228,3239,3273,3275,3276,3279,...,33138,33145,33148,33150,33152,33154,33158,33162,33164,33166
2769,1.000000,0.115068,0.033721,-0.232268,NaN,-0.500000,0.197011,0.199514,0.250873,NaN,...,0.379980,0.878310,NaN,NaN,NaN,0.248126,0.180609,-0.085570,-0.408248,0.105671
3177,0.115068,1.000000,0.303820,0.559533,NaN,NaN,0.331191,0.167918,1.000000,NaN,...,0.546119,0.735767,-1.0,NaN,NaN,-0.221382,0.317475,0.014735,0.661989,0.185654
3190,0.033721,0.303820,1.000000,0.636361,NaN,-0.014315,0.146042,0.394293,-0.290397,NaN,...,0.246183,0.632026,NaN,NaN,NaN,0.378181,0.170926,0.022444,-0.073360,-0.054114
3225,-0.232268,0.559533,0.636361,1.000000,NaN,0.578414,0.347716,0.263671,-0.250313,NaN,...,-0.300376,0.318377,NaN,NaN,NaN,0.480173,0.750306,0.536828,0.753141,0.098748
3228,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# code to compute all correlations between columns (movies) in the matrix user_movie, 
# but considering only movies having at least 100 ratings in common. 
# Store this in "item_similarity_min_ratings"

item_similarity_min_ratings = ratings_selected.corr(min_periods=100)

item_similarity_min_ratings.head(5)

,2769,3177,3190,3225,3228,3239,3273,3275,3276,3279,...,33138,33145,33148,33150,33152,33154,33158,33162,33164,33166
2769,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3177,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3190,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3225,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
super_moviesIds = [5349, 3793, 6534]
drama_moviesIds = [6870, 5995, 3555]

# code to find the userids of two example users: user_id_super (the who liked the three superhero movies), 
# and user_id_drama (the one who liked the three dramas)
def find_user_IDS(movieIDS, min_rate, df):

  selection = df[movieIDS]

  filter = None

  c = 0

  for id in movieIDS:
    if c == 0:
      filter = (selection[id] > min_rate)
    
    else:
      filter = (filter) & (selection[id] > min_rate)

    c += 1

  selection = selection.loc[filter]



  
  selection = selection.dropna()
  
  selection.to_csv
  print(selection)

  return selection.index


In [34]:
user_id_super = find_user_IDS(super_moviesIds, 4.5, user_movie)

movie_id  5349  3793  6534
user_id                   
127342     5.0   5.0   5.0


In [35]:
user_id_drama = find_user_IDS(drama_moviesIds, 4.5, user_movie)[0]

movie_id  6870  5995  3555
user_id                   
34336      5.0   5.0   5.0
68770      5.0   5.0   5.0
147353     5.0   5.0   5.0


In [36]:
# Leave this code as-is

# Gets a list of watched movies for a user_id
def get_watched_movies(user_id, user_movie):
    mov = user_movie.loc[user_id]

    if isinstance(mov, pd.DataFrame):
      mov = mov.squeeze()

      
    movies_watched = mov.dropna()
    
    return movies_watched.index

    
    
    
# Gets the title of a movie by movie_id
def get_title(movie_id, movies):
    
    return movies[movies['movie_id'] == movie_id].title.iloc[0]

# Gets the rating a user_id has given to a movie_id
def get_rating(user_id, movie_id, user_movie):
    
    return user_movie[movie_id][user_id]

# Print watched movies
def print_watched_movies(user_id, user_movie, movies):
    for movie_id in get_watched_movies(user_id, user_movie):
        print("%d %.1f %s " %
          (movie_id, get_rating(user_id, movie_id, user_movie), get_title(movie_id, movies)))


In [37]:
# LEAVE AS-IS (TESTING CODE)

print_watched_movies(user_id_super, user_movie, movies)

3300 0.5 Pitch Black (2000) 
3354 4.5 Mission to Mars (2000) 
3408 4.5 Erin Brockovich (2000) 
3481 4.5 High Fidelity (2000) 
3510 3.5 Frequency (2000) 
3555 4.0 U-571 (2000) 
3564 3.5 Flintstones in Viva Rock Vegas, The (2000) 
3578 5.0 Gladiator (2000) 
3593 2.5 Battlefield Earth (2000) 
3598 5.0 Hamlet (2000) 
3623 5.0 Mission: Impossible II (2000) 
3624 5.0 Shanghai Noon (2000) 
3744 4.0 Shaft (2000) 
3745 4.0 Titan A.E. (2000) 
3751 5.0 Chicken Run (2000) 
3753 5.0 Patriot, The (2000) 
3755 5.0 Perfect Storm, The (2000) 
3785 5.0 Scary Movie (2000) 
3793 5.0 X-Men (2000) 
3826 4.0 Hollow Man (2000) 
3827 5.0 Space Cowboys (2000) 
3863 5.0 Cell, The (2000) 
3889 4.0 Highlander: Endgame (Highlander IV) (2000) 
3977 4.0 Charlie's Angels (2000) 
3981 4.5 Red Planet (2000) 
3986 3.0 6th Day, The (2000) 
3988 5.0 How the Grinch Stole Christmas (a.k.a. The Grinch) (2000) 
3996 4.0 Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000) 
3997 4.0 Dungeons & Dragons (2000) 
3998 4.0 Proof 

In [38]:
# LEAVE AS-IS (TESTING CODE)

print_watched_movies(user_id_drama, user_movie, movies)

3510 4.5 Frequency (2000) 
3555 5.0 U-571 (2000) 
3578 2.5 Gladiator (2000) 
3752 1.0 Me, Myself & Irene (2000) 
3755 4.0 Perfect Storm, The (2000) 
3897 4.0 Almost Famous (2000) 
3916 2.0 Remember the Titans (2000) 
3948 3.5 Meet the Parents (2000) 
3967 5.0 Billy Elliot (2000) 
4014 5.0 Chocolat (2000) 
4018 2.5 What Women Want (2000) 
4019 3.5 Finding Forrester (2000) 
4022 4.0 Cast Away (2000) 
4025 2.5 Miss Congeniality (2000) 
4027 2.5 O Brother, Where Art Thou? (2000) 
4034 5.0 Traffic (2000) 
4246 3.5 Bridget Jones's Diary (2001) 
4308 4.0 Moulin Rouge (2001) 
4310 3.0 Pearl Harbor (2001) 
4447 3.5 Legally Blonde (2001) 
4776 2.5 Training Day (2001) 
4963 2.5 Ocean's Eleven (2001) 
4975 3.5 Vanilla Sky (2001) 
4993 3.0 Lord of the Rings: The Fellowship of the Ring, The (2001) 
4995 5.0 Beautiful Mind, A (2001) 
5010 4.0 Black Hawk Down (2001) 
5015 4.5 Monster's Ball (2001) 
5299 4.0 My Big Fat Greek Wedding (2002) 
5349 3.0 Spider-Man (2002) 
5377 3.5 About a Boy (2002) 
5418 

In [39]:
# code for "get_movies_relevance"
def get_movies_relevance(user_id, user_movie, item_similarity_matrix):

    # Create an empty series
    movies_relevance = pd.Series()
    
    # Iterate through the movies the user has watched
    for watched_movie in get_watched_movies(user_id, user_movie):

        # Obtain the rating given
        rating_given = get_rating(user_id, watched_movie, user_movie)
        
        # Obtain the vector containing the similarities of watched_movie
        # with all other movies in item_similarity_matrix
        similarities = item_similarity_matrix[watched_movie]
        
        # Multiply this vector by the given rating
        weighted_similarities =  float(rating_given) * similarities  
        
        # Append these terms to movies_relevance
        movies_relevance = movies_relevance.append(weighted_similarities)
        
    # Compute the sum for each movie
    movies_relevance = movies_relevance.groupby(movies_relevance.index).sum()

    # Convert to a dataframe
    movies_relevance_df = pd.DataFrame(movies_relevance, columns=['relevance'])
    movies_relevance_df['movie_id'] = movies_relevance_df.index
    
    return movies_relevance_df

In [40]:
# code to obtain the 10 most relevant movies for the users user_id_super (who likes superhero movies) 
super_user = get_movies_relevance(user_id_super , user_movie, item_similarity_min_ratings)

super_user = pd.merge(super_user, movies, how='inner', on='movie_id')

super_user = super_user.sort_values(by= 'relevance', ascending=False)

super_user = super_user.set_index('movie_id', drop=False)

In [41]:
# code to obtain the 10 most relevant movies for the user_id_drama (who likes dramas)
drama_user = get_movies_relevance(user_id_drama, user_movie, item_similarity_min_ratings)

drama_user = pd.merge(drama_user, movies, how='inner', on='movie_id')

drama_user = drama_user.sort_values(by= 'relevance', ascending=False)
drama_user = drama_user.set_index('movie_id', drop=False)

In [42]:
super_user.head(10)

,relevance,movie_id,title,genres
movie_id,,,,
8644,189.170085,8644,"I, Robot (2004)",Action|Adventure|Sci-Fi|Thriller
5459,181.638120,5459,Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (...,Action|Comedy|Sci-Fi
3753,176.650945,3753,"Patriot, The (2000)",Action|Drama|War
8361,172.899804,8361,"Day After Tomorrow, The (2004)",Action|Adventure|Drama|Sci-Fi|Thriller
4310,172.700877,4310,Pearl Harbor (2001),Action|Drama|Romance|War
4270,172.301301,4270,"Mummy Returns, The (2001)",Action|Adventure|Comedy|Thriller
4367,169.123776,4367,Lara Croft: Tomb Raider (2001),Action|Adventure
6373,168.960164,6373,Bruce Almighty (2003),Comedy|Drama|Fantasy|Romance
8972,168.783883,8972,National Treasure (2004),Action|Adventure|Drama|Mystery|Thriller


In [43]:
drama_user.head(10)

,relevance,movie_id,title,genres
movie_id,,,,
8958,65.461370,8958,Ray (2004),Drama
4019,63.007635,4019,Finding Forrester (2000),Drama
6565,61.354376,6565,Seabiscuit (2003),Drama
4995,61.213050,4995,"Beautiful Mind, A (2001)",Drama|Romance
5014,61.209632,5014,I Am Sam (2001),Drama
7143,60.751048,7143,"Last Samurai, The (2003)",Action|Adventure|Drama|War
8644,60.700299,8644,"I, Robot (2004)",Action|Adventure|Sci-Fi|Thriller
6870,60.611768,6870,Mystic River (2003),Crime|Drama|Mystery
7325,59.820898,7325,Starsky & Hutch (2004),Action|Comedy|Crime|Thriller


From my knowledge in the first case we can see clearly that the recomendations are accurate regarding the previous ratings of our user. In the second case I don't know too much because I didn't see a lot of drama movies.

In [44]:
# code implementing "get_recommended_movies"
def get_recommended_movies(user_id, user_movie, relevant_movies):
    whatched = get_watched_movies(user_id, user_movie)

    relevant = relevant_movies.drop(whatched, axis=0)
    return relevant

In [45]:
# code to obtain the 10 most recommended movies for the user_id_super
super_user_recommended = get_recommended_movies(user_id_super, user_movie, super_user)

In [46]:
# # code to obtain the 10 most recommended movies for the user_id_drama
drama_user_recommended = get_recommended_movies(user_id_drama, user_movie, drama_user)

In [47]:
super_user_recommended.head(10)

,relevance,movie_id,title,genres
movie_id,,,,
6365,166.866641,6365,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX
4018,165.338077,4018,What Women Want (2000),Comedy|Romance
4025,163.032765,4025,Miss Congeniality (2000),Comedy|Crime
5507,161.080324,5507,xXx (2002),Action|Crime|Thriller
6378,155.293219,6378,"Italian Job, The (2003)",Action|Crime
31685,154.993274,31685,Hitch (2005),Comedy|Romance
3948,150.570934,3948,Meet the Parents (2000),Comedy
4369,148.949754,4369,"Fast and the Furious, The (2001)",Action|Crime|Thriller
6934,148.394158,6934,"Matrix Revolutions, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX


In [48]:
drama_user_recommended.head(10)

,relevance,movie_id,title,genres
movie_id,,,,
8958,65.461370,8958,Ray (2004),Drama
6565,61.354376,6565,Seabiscuit (2003),Drama
5014,61.209632,5014,I Am Sam (2001),Drama
7325,59.820898,7325,Starsky & Hutch (2004),Action|Comedy|Crime|Thriller
7149,59.294621,7149,Something's Gotta Give (2003),Comedy|Drama|Romance
4448,58.968024,4448,"Score, The (2001)",Action|Drama
7445,58.192646,7445,Man on Fire (2004),Action|Crime|Drama|Mystery|Thriller
5152,58.004447,5152,We Were Soldiers (2002),Action|Drama|War
3753,57.920754,3753,"Patriot, The (2000)",Action|Drama|War


I believe that these two lists that recommend movies for the super user and the drama user are fairly accurate.  After removing the movies that the user has watched, the relevance scores remain comparable, even though a bit lower (meaning that the user has watched already some of those which were recommended at first), and therefore it is a good recommendating system.



# Extra points



In [49]:
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import BaselineOnly
from surprise import accuracy

In [50]:
# Surprise reader
reader = Reader(rating_scale=(0, 5))

# Load all ratings
ratings = Dataset.load_from_df(ratings[['user_id', 'movie_id',  'rating' ]], reader)

In [51]:
train_ratings, test_ratings = train_test_split(ratings, test_size=.20, random_state = 40)

In [52]:
baseline_model = BaselineOnly(verbose = False)
baseline_model.fit(train_ratings)

In [53]:
train_predictions = baseline_model.test(train_ratings.build_testset())
test_predictions = baseline_model.test(test_ratings)

In [54]:
def get_top_n_recommendations(userId,predictions, n):
    predict_ratings = {}
    # loop for getting predictions for the user
    for uid, iid, true_r, est, _ in predictions:
        if (uid==userId):
            predict_ratings[iid] = est
    predict_ratings = sorted(predict_ratings.items(), key=lambda kv: kv[1],reverse=True)[:n]

    top_movies = [i[0] for i in predict_ratings]
  
    print("="*10,f"Recommended movies for user {(userId)}","="*10)

    for movie_id in top_movies:
      print(get_title(movie_id, movies)) 

In [55]:
get_top_n_recommendations(user_id_super, test_predictions, 10)

========== Recommended movies for user Int64Index([127342], dtype='int64', name='user_id') ==========
Buffalo Soldiers (2001)
X2: X-Men United (2003)
Guerrilla: The Taking of Patty Hearst (2004)
Open Range (2003)
Shrek 2 (2004)
Pitch Black (2000)
Solaris (2002)
Signs (2002)
Final Fantasy: The Spirits Within (2001)
Cell, The (2000)


In [56]:
get_top_n_recommendations(user_id_drama, test_predictions, 10)

========== Recommended movies for user 34336 ==========
Lord of the Rings: The Fellowship of the Ring, The (2001)
Gladiator (2000)
O Brother, Where Art Thou? (2000)
Kill Bill: Vol. 2 (2004)
Finding Forrester (2000)
I, Robot (2004)
Vanilla Sky (2001)
Miss Congeniality (2000)
Pearl Harbor (2001)
